In [8]:
def getpreppedinputs( dev_texts, dev_labels,max_length=300, by_sentence=True):
    nlp = spacy.load('en_vectors_web_lg')
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    embeddings = get_embeddings(nlp.vocab)

    
    #model = compile_lstm(embeddings, lstm_shape, lstm_settings)
    print("Parsing texts...")
    #train_docs = list(nlp.pipe(train_texts))
    dev_docs = list(nlp.pipe(dev_texts))
    if by_sentence:
        #train_docs, train_labels = get_labelled_sentences(train_docs, train_labels)
        dev_docs, dev_labels = get_labelled_sentences(dev_docs, dev_labels)

    #train_X = get_features(train_docs, lstm_shape['max_length'])
    dev_X = get_features(dev_docs, max_length)
    return dev_X,dev_labels


#Dont need to use get_labelled_sentences if Im using readALlfiles
def readAllFiles(dirpath='Call-Abstractor/fullText',max_length=300):
    sentences=[]
    labels=[]
    for file in glob.glob(path):
        text=open(file,'r').read()
        doc=nlp(text)
        for sent in doc.sents:
            sentences.append(sent.text)
            labels.append(random.randint(0,4))
    sentences=get_features(sentences,max_length)
    return sentences,numpy.asarray(labels,dtype='int32')
        

NameError: name 'rand' is not defined

In [ ]:
train_texts, train_labels =read_AllFiles(dirpath='Call-Abstractor/taggedCalls/2Calls.csv')
from sklearn.model_selection import train_test_split

#data, labels = np.arange(10).reshape((5, 2)), range(5)

train_texts, dev_texts, train_labels, dev_labels = train_test_split(train_texts, train_labels, test_size=0.20)
lstm = train(train_texts, train_labels, dev_texts, dev_labels,
                 {'nr_hidden': nr_hidden, 'max_length': max_length, 'nr_class': 5},
                 {'dropout': dropout, 'lr': learn_rate},
                 {},
                 nb_epoch=nb_epoch, batch_size=batch_size)
weights = lstm.get_weights()
lstm.save('Call-Abstractor/model/imdbcaller.h5')

In [1]:
"""
This example shows how to use an LSTM sentiment classification model trained using Keras in spaCy. spaCy splits the document into sentences, and each sentence is classified using the LSTM. The scores for the sentences are then aggregated to give the document score. This kind of hierarchical model is quite difficult in "pure" Keras or Tensorflow, but it's very effective. The Keras example on this dataset performs quite poorly, because it cuts off the documents so that they're a fixed size. This hurts review accuracy a lot, because people often summarise their rating in the final sentence
Prerequisites:
spacy download en_vectors_web_lg
pip install keras==2.0.9
Compatible with: spaCy v2.0.0+
"""
import spacy
import re
from sense2vec import Sense2VecComponent
nlps2v = spacy.load('en')  # make sure to use larger model!
s2v=Sense2VecComponent('reddit_vectors-1.1.0')
nlps2v.add_pipe(s2v)
nlp = spacy.load('en_vectors_web_lg')
nlp.add_pipe(nlp.create_pipe('sentencizer'))

import getFinSent
import plac
import random,pickle
import pathlib
import cytoolz
from keras.utils import to_categorical
import numpy
from keras.preprocessing import sequence
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import thinc.extra.datasets
from spacy.compat import pickle
import spacy
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
import glob
from random import randint
posList=getFinSent.loadPositive()
negList=getFinSent.loadNegative()
class SentimentAnalyser(object):
    @classmethod
    def load(cls, path, embeddings,nlp, max_length=100):
        """if path!="default":
            path=('../TransLearn/models')"""
        with (path / 'config.json').open() as file_:
            model = model_from_json(file_.read())
        with (path / 'model').open('rb') as file_:
            lstm_weights = pickle.load(file_)
        if embeddings is None:
            embeddings = get_embeddings(nlp.vocab)
        model.set_weights([embeddings] + lstm_weights)
        return cls(model, max_length=max_length)

    def __init__(self, model, max_length=100):
        self._model = model
        self.max_length = max_length

    def __call__(self, doc):
        X = get_features([doc], self.max_length)
        y = self._model.predict(X)
        self.set_sentiment(doc, y)

    def pipe(self, docs, batch_size=1000, n_threads=2):
        for minibatch in cytoolz.partition_all(batch_size, docs):
            minibatch = list(minibatch)
            sentences = []
            for doc in minibatch:
                sentences.extend(doc.sents)
            Xs = get_features(sentences, self.max_length)
            ys = self._model.predict(Xs)
            for sent, label in zip(sentences, ys):
                sent.doc.sentiment += label - 0.5
            for doc in minibatch:
                yield doc

    def set_sentiment(self, doc, y):
        doc.sentiment = float(y[0])
        # Sentiment has a native slot for a single float.
        # For arbitrary data storage, there's:
        # doc.user_data['my_data'] = y

        


def get_labelled_sentences(docs, doc_labels):
    
    #This function has been changed from the original
    labels = []
    sentences = []
    for doc, y in zip(docs, doc_labels):
        
        sentences.append(doc)
        if y<-1:
            onehot=0
        elif y<0:
            onehot=1
        elif y==0:
            onehot=2
        elif y<=1:
            onehot=3
        elif y>1:
            onehot=4
        else:
            print('Error while encoding label')
                
            
        labels.append(onehot)
    newlabels=to_categorical(labels,5)
    #print('Some labels are'+str(newlabels[:20]))
            
    #return sentences, numpy.asarray(labels, dtype='int32')
    return sentences, newlabels
#Return matrix representation of all sentences embedded with token ids. Sentence will look like [1,192,3,6...]


def get_features(docs, max_length):
    docs = list(docs)
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    c=0
    #doc is a string. A sentence text
    for i, doc in enumerate(docs):
        j = 0
        if c<30:
            c+=1
            #print("Inside get_features. doc is"+str(doc)+" \n Type of doc is"+str(type(doc)))
        breakdoc=nlp(doc)
        for token in breakdoc:
            #print(type(token))
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    #print("Inside get_features. Some vector IDS")
    print(Xs[0, 10])
    print(Xs[1,39])
    
    return Xs


def compile_lstm(embeddings, shape, settings,trainable=True):
    model = Sequential()
    model.add(
        Embedding(
            embeddings.shape[0],
            embeddings.shape[1],
            input_length=shape['max_length'],
            trainable=trainable,
            weights=[embeddings],
            mask_zero=True
        )
    )
    model.add(TimeDistributed(Dense(shape['nr_hidden'], use_bias=False)))
    model.add(Bidirectional(LSTM(shape['nr_hidden'],
                                 recurrent_dropout=settings['dropout'],
                                 dropout=settings['dropout'])))
    model.add(Dense(shape['nr_class'], activation='softmax'))
    model.compile(optimizer=Adam(lr=settings['lr']), loss='categorical_crossentropy',
          metrics=['accuracy'])
    model.summary()
    return model,embeddings


def get_embeddings(vocab):
    return vocab.vectors.data


def evaluate(model_dir, texts, labels, max_length=100):
    def create_pipeline(nlp):
        '''
        This could be a lambda, but named functions are easier to read in Python.
        '''
        return [nlp.tagger, nlp.parser, SentimentAnalyser.load(model_dir, nlp,
                                                               max_length=max_length)]

    """nlp = spacy.load('en')
    nlp.pipeline = create_pipeline(nlp)"""

    correct = 0
    i = 0
    for doc in nlp.pipe(texts, batch_size=1000, n_threads=4):
        correct += bool(doc.sentiment >= 0.5) == bool(labels[i])
        i += 1
    return float(correct) / i

#Only used for the initial phase of Transfer Learning
def read_data(data_dir, limit=0):
    examples = []
    """for subdir, label in (('pos', 1), ('neg', 0)):
        for filename in (data_dir / subdir).iterdir():
            with filename.open() as file_:
                text = file_.read()
            examples.append((text, label))
    random.shuffle(examples)"""
    c=0
    #nlp.add_pipe(s2v)
    #print(data_dir)
    texts=[]
    for filename in glob.glob(data_dir+"*.txt"):
        #print(filename)
        file=open(filename,'r')
        ftext = file.read()
        #==================
        texts.append(ftext)
        

        
        #=====================
        docs=nlps2v(ftext)
        for sent in docs.sents:
            stext=sent.text
            cleanText=[]
            s2vsent = nlps2v(stext)
            # Break sentence into list of tokens
            for token in s2vsent:
                cleanText.append(token.text)
            label=getFinSent.getSentiment(cleanText,negList,posList)
            #print("Inside read_data"+str(label))
            #if label!=0:
                
            #ds
            #print(stext+'!!!!!!!!!!!! Label'+str(label))
            examples.append((stext,label))
            
    #nlp.remove_pipe('sense2vec')
    """if limit >= 1:
        examples = examples[:]"""
    #======
    
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    MAX_NB_WORDS=50000
    tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    #======
    #print("Inside read_data,  examples"+str(examples))
    return zip(*examples) # Unzips into two lists

def train(train_texts, train_labels, dev_texts, dev_labels,
          lstm_shape, lstm_settings, lstm_optimizer, batch_size=100,
          nb_epoch=5, by_sentence=True,model=None,embeddings=None,trainable=True,labels="-5to5",usinginputs=False):
    print("Loading spaCy")
    
    
    if model is None and embeddings is None:
        
        embeddings = get_embeddings(nlp.vocab)
        print("embeddings shape is "+str(embeddings.shape))
        model,embeddings = compile_lstm(embeddings, lstm_shape, lstm_settings,trainable=trainable)
    if not usinginputs:
        print("Parsing texts...")

        #train_docs = list(nlp.pipe(train_texts))
        #dev_docs = list(nlp.pipe(dev_texts))

        train_docs=train_texts
        #dev_docs=dev_texts
        #print("Inside train, Length of train_texts is"+str(len(train_texts)))
        print("One sample is "+str(train_texts[0])+" And type is"+str(type(train_texts[0])))
        #print("Labels are"+str(train_labels))
        if by_sentence:
            if(labels=="-5to5"):
                train_docs, train_labels = get_labelled_sentences(train_texts, train_labels)
                #dev_docs, dev_labels = get_labelled_sentences(dev_texts, dev_labels)
            elif(labels=="0to1"):
                pass
            else:
                train_labels=list(train_labels)
                train_labels=to_categorical(train_labels,5)
                #Assume the labeling is already in 0to4 format
        print("Received train_labels from get functions  are "+str(train_labels))
        print("Inside train, Length of train_docs is"+str(len(train_docs)))
        train_X = get_features(train_docs, lstm_shape['max_length'])

        #dev_X = get_features(dev_docs, lstm_shape['max_length'])
        print("Inside train, Shape of dev_X is"+str(dev_X.shape))
        print("One devX sampe is "+str(dev_X[0]))
        print(train_X.shape)
        print(train_labels.shape)
        print('Train_labels 2 of them are'+str(train_labels[:2]))
        print('devlabels 2 of them are'+str(dev_labels[:2]))
        print('Inside train...')
        #print(train_X[0])
        #print("Labels are"+str(train_labels))
        pickle.dump(train_X,open("../TransLearn/2calls/bwgbctrain_X.p","wb"))
        pickle.dump(train_labels,open("../TransLearn/2calls/bwgbctrain_labels.p","wb"))
        pickle.dump(dev_X,open("../TransLearn/2calls/bwgbcdev_X.p","wb"))
        pickle.dump(dev_labels,open("../TransLearn/2calls/bwgbcdev_labels.p","wb"))
    else:
        train_X=pickle.load(open("../TransLearn/2calls/bctrain_X.p","rb"))
        train_labels=pickle.load(open("../TransLearn/2calls/bctrain_labels.p","rb"))
    history=model.fit(train_X, train_labels, validation_split=0.3,
              nb_epoch=nb_epoch, batch_size=batch_size)
    
    print('Shape of embeddings'+str(embeddings.shape))
    pickle.dump(embeddings,open('../TransLearn/2calls/BWGRotEmbeddings.p','wb'))
    return model,history





/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model_dir=None
train_dir=None
dev_dir=None
is_runtime=False
nr_hidden=256
max_length=50; # Shape
dropout=0.5
learn_rate=0.001; # General NN config
nb_epoch=2
batch_size=64
nr_examples=2000 # Training params
"""if model_dir is not None:
    model_dir = pathlib.Path(model_dir)
if train_dir is None or dev_dir is None:
    imdb_data = thinc.extra.datasets.imdb()
if is_runtime:
    if dev_dir is None:
        dev_texts, dev_labels = zip(*imdb_data[1])
    else:
        dev_texts, dev_labels = read_data(dev_dir)
    acc = evaluate(model_dir, dev_texts, dev_labels, max_length=max_length)
    print(acc)
else:"""
train_dir="fullText/";dev_dir="dev_dir/";model_dir="glove/"


if train_dir is None:
    train_texts, train_labels = zip(*imdb_data[0])
else:
    print("Read data")
    #Get a list of Spacy sent objects intact and unbroken and their labels
    train_texts, train_labels = read_data(train_dir, limit=nr_examples)
    print("Inside main function, labels are"+str(type(train_labels)))
if dev_dir is None:
    dev_texts, dev_labels = zip(*imdb_data[1])
else:
    dev_texts, dev_labels = read_data(dev_dir, limit=nr_examples)



Read data


/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/preprocessing/text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 23319 unique tokens.
Inside main function, labels are<class 'tuple'>
Found 15744 unique tokens.


In [ ]:
#train_labels = numpy.asarray(train_labels, dtype='int32')
#dev_labels = numpy.asarray(dev_labels, dtype='int32')
lstm,history = train(train_texts, train_labels, dev_texts, dev_labels,
             {'nr_hidden': nr_hidden, 'max_length': max_length, 'nr_class': 5},
             {'dropout': dropout, 'lr': learn_rate},
             {},
             nb_epoch=nb_epoch, batch_size=batch_size,by_sentence=True)
weights = lstm.get_weights()
lstm.save('../TransLearn/models/BWGTLModel.p')
lstm.save_weights('../TransLearn/BWGTLModelweights.h5')
"""if model_dir is not None:
    with (model_dir / 'model').open('wb') as file_:
        pickle.dump(weights[1:], file_)
    with (model_dir / 'config.json').open('wb') as file_:
        file_.write(lstm.to_json())"""

Loading spaCy
embeddings shape is (1070971, 300)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           321291300 
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 256)           76800     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1050624   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2565      
Total params: 322,421,289
Trainable params: 322,421,289
Non-trainable params: 0
_________________________________________________________________
Parsing texts...
One sample is welcome to the bwg strategy Conference Center please enter your user ID the conference will be getting it when the host arrives the conference is now in presentation mode you're ly

/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '
/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 321291300 elements. This may consume a large amount of memory.
  num_elements)


Train on 50407 samples, validate on 21603 samples
Epoch 1/2
24576/50407 [=============>................] - ETA: 36:22 - loss: 0.4402 - acc: 0.8672

In [ ]:
import json
lstmjson=lstm.to_json()
with open('../TransLearn/models/PTconfig.json', 'w') as outfile:
    json.dump(jsonstring, outfile)

In [10]:
"""def getpickles():
    train_X=pickle.load(open("glove/train_X.p",'rb'))
    dev_X=pickle.load(open("glove/dev_X.p",'rb'))
    train_labels=pickle.load(open("glove/train_labels.p",'rb'))
    dev_labels=pickle.load(open("glove/dev_labels.p","rb"))
    return train_X,dev_X,train_labels,dev_labels

train_X,dev_X,train_labels,dev_labels=getpickles()
embeddings = get_embeddings(nlp.vocab)
model,embeddings = compile_lstm(embeddings, {'nr_hidden': nr_hidden, 'max_length': max_length, 'nr_class': 5},
             {'dropout': dropout, 'lr': learn_rate}
             )
newhistory=model.fit(train_X, train_labels, validation_split=0.3,
              nb_epoch=2, batch_size=batch_size)
    """

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 300)          321291300 
_________________________________________________________________
time_distributed_4 (TimeDist (None, 100, 128)          38400     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 1285      
Total params: 321,594,153
Trainable params: 321,594,153
Non-trainable params: 0
_________________________________________________________________


/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '
/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 321291300 elements. This may consume a large amount of memory.
  num_elements)


Train on 50407 samples, validate on 21603 samples
Epoch 1/2
50407/50407 [==============================] - 3832s 76ms/step - loss: 0.3477 - acc: 0.8862 - val_loss: 0.1825 - val_acc: 0.9227
Epoch 2/2
50407/50407 [==============================] - 3826s 76ms/step - loss: 0.1473 - acc: 0.9366 - val_loss: 0.1361 - val_acc: 0.9503


In [ ]:
import matplotlib.pyplot as plt
plt.plot(newhistory.history['acc'])
plt.plot(newhistory.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [16]:

model.save('../TransLearn/models/transmodel1')
model.save_weights('../Translearn/transmodel1_weights.h5')

In [14]:
newmodel,embeddings=compileLSTM()

In [2]:
import pandas as pd
def read_dataFrame(dirpath):
    #A substitute for read_data function.
    dflist=[]
    for filename in glob.glob(dirpath+"train.tsv"):
        dflist.append(pd.read_csv(filename,sep='\t',header=0))
    traindf=pd.concat(dflist)
    traindf.describe()
    examples=[]
    for indx,row in traindf.iterrows():
        label=row['Sentiment']
        sentence=row['Phrase']
        examples.append((sentence,label))
    #print(traindf.columns)
    #labels=traindf['Sentiment'].values
    #train_labels=to_categorical(numpy.asarray(labels),num_classes=5)
    return zip(*examples)
    #return traindf
    
    
def TransferLearn(embeddings,trainable,labels,model=None,dirpath="taggedCalls/",usinginputs=False):
    #oldmodel=keras.model.load_model("modelpath")
    model_dir=None
    train_dir=None
    dev_dir=None
    is_runtime=False
    nr_hidden=256
    max_length=50; # Shape
    dropout=0.5
    learn_rate=0.001; # General NN config
    nb_epoch=3
    batch_size=64
    nr_examples=2000
    if not usinginputs:
        train_texts,train_labels=read_dataFrame(dirpath)
    else:
        train_texts="Scenes bro no input"
        train_labels="Scenes bro no labels"
    model.summary()
    bwglstm,bwghistory = train(train_texts, train_labels,"","", 
             {'nr_hidden': nr_hidden, 'max_length': max_length, 'nr_class': 5},
             {'dropout': dropout, 'lr': learn_rate},
             {},
                         
             nb_epoch=nb_epoch, batch_size=batch_size,by_sentence=True,model=model,embeddings=embeddings,trainable=trainable,labels=labels,usinginputs=usinginputs)
    return lstm,history

In [3]:
#My Attempt to train for rotomato

import keras
embeddings=pickle.load(open('../TransLearn/2calls/TLembeddings.p','rb'))
bwgbasemodel=keras.models.load_model("../TransLearn/models/TLModel.p")
for layer in bwgbasemodel.layers:
    layer.trainable=False
bwgbasemodel.layers[-1].trainable=True
bwlstm,history=TransferLearn(model=bwgbasemodel,dirpath="rottentomatoes/data/sentiment/",
                             embeddings=embeddings,trainable=False,labels="0to4",usinginputs=True)
bwlstm.save('../TransLearn/models/rotomato_frozenlayers')
bwlstm.save_weight('../TransLearn/models/rotomato_frozenlayerwts.h5')

/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 321291300 elements. This may consume a large amount of memory.
  num_elements)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           321291300 
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 256)           76800     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1050624   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2565      
Total params: 322,421,289
Trainable params: 2,565
Non-trainable params: 322,418,724
_________________________________________________________________
Loading spaCy
Train on 109242 samples, validate on 46818 samples
Epoch 1/3


/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '
/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


109242/109242 [==============================] - 7750s 71ms/step - loss: 0.8506 - acc: 0.6477 - val_loss: 0.8993 - val_acc: 0.6253
Epoch 2/3
109242/109242 [==============================] - 7281s 67ms/step - loss: 0.7163 - acc: 0.7020 - val_loss: 0.9288 - val_acc: 0.6211
Epoch 3/3
 72832/109242 [===================>..........] - ETA: 40:20 - loss: 0.6488 - acc: 0.7279

KeyboardInterrupt: 

In [13]:
"""import keras
embeddings=pickle.load(open('../TransLearn/2calls/TLembeddings.p','rb'))
bwgbasemodel=keras.models.load_model("../TransLearn/models/TLModel.p")

lstm,history=TransferLearn(model=bwgbasemodel,dirpath="taggedCalls/",embeddings=embeddings,trainable=True,labels="0to4")"""

/Users/prashanththekkada/.virtualenvs/dl4cv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 321291300 elements. This may consume a large amount of memory.
  num_elements)


NameError: name 'dev_texts' is not defined

In [ ]:
def readTomatoes(dirpath):
    traindf = pd.read_csv('rottentomatoes/train.tsv', sep='\t', header=0)
    examples=[]
    for indx,row in traindf.iterrows():
        label=row['Sentiment']
        sentence=row['Phrase']
        examples.append((sentence,label))
    #print(traindf.columns)
    #labels=traindf['Sentiment'].values
    #train_labels=to_categorical(numpy.asarray(labels),num_classes=5)
    return zip(*examples)
    #return traindf
def readLMDict(dirpath):
    

In [3]:
print(getFinSent.getSentiment("rise with improvements in increase",negList,posList))

0


In [5]:
posList

['able',
 'abundance',
 'abundant',
 'acclaimed',
 'accomplish',
 'accomplished',
 'accomplishes',
 'accomplishing',
 'accomplishment',
 'accomplishments',
 'achieve',
 'achieved',
 'achievement',
 'achievements',
 'achieves',
 'achieving',
 'adequately',
 'advancement',
 'advancements',
 'advances',
 'advancing',
 'advantage',
 'advantaged',
 'advantageous',
 'advantageously',
 'advantages',
 'alliance',
 'alliances',
 'assure',
 'assured',
 'assures',
 'assuring',
 'attain',
 'attained',
 'attaining',
 'attainment',
 'attainments',
 'attains',
 'attractive',
 'attractiveness',
 'beautiful',
 'beautifully',
 'beneficial',
 'beneficially',
 'benefit',
 'benefited',
 'benefiting',
 'benefitted',
 'benefitting',
 'best',
 'better',
 'bolstered',
 'bolstering',
 'bolsters',
 'boom',
 'booming',
 'boost',
 'boosted',
 'breakthrough',
 'breakthroughs',
 'brilliant',
 'charitable',
 'collaborate',
 'collaborated',
 'collaborates',
 'collaborating',
 'collaboration',
 'collaborations',
 'coll

In [ ]:
import s

In [6]:
g=nlp('hey there. My name is goku. I am a disco dancer')
for token in g:
    print(token.vocab.vectors.find(key=token.orth))
    print(token)
print("="*25)
temp=nlp(g.text)
for token in temp:
    print(token.vocab.vectors.find(key=token.orth))
    print(token)

177
hey
230
there
79
.
477
My
662
name
137
is
759897
goku
79
.
132
I
398
am
130
a
135240
disco
78455
dancer
177
hey
230
there
79
.
477
My
662
name
137
is
759897
goku
79
.
132
I
398
am
130
a
135240
disco
78455
dancer


In [7]:
from spacy.tokens import Token
Token.set_extension('force', True)

TypeError: set_extension() takes exactly 1 positional argument (2 given)

In [6]:
k="hey,there he. kill's me."
k.split(' .,')

["hey,there he. kill's me."]

In [8]:
from keras.utils import to_categorical
type(to_categorical([3,2,1],4))

numpy.ndarray